<a href="https://colab.research.google.com/github/Hiroki31730/Seminar/blob/main/CNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Google colabではローカル環境にアクセスできないのでdiveと接続できるようにする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# インポート
import pandas as pd
import numpy as np
import os
import random
import shutil
import glob

# CNN用インポート
import keras
from keras.preprocessing import image
from keras.utils import to_categorical
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array

In [3]:
# 学習データとテストデータに分割
D_path = '/content/drive/MyDrive'

def Test_Set_CNN(rate): # rate:分割の割合
    dir_ph = os.path.join(D_path, 'SANSIN_sp2')   # 元データのディレクトリパス
    dir_Cnnph = os.path.join(D_path, 'CNN_sp_data')  # テストデータ、学習データの親ディレクトリパス
    spl_tstph = os.path.join(dir_Cnnph, 'CNN_test_data') # テストデータディレクトリパス
    spl_lrnph = os.path.join(dir_Cnnph, 'CNN_train_data') # 学習データディレクトリパス

    # すでにディレクトリ(dir_Cnnph)が存在していれば全て削除
    if os.path.isdir(dir_Cnnph):
        shutil.rmtree(dir_Cnnph)

    # ディレクトリ作成
    os.makedirs(spl_tstph, exist_ok=True) # テストデータ
    os.makedirs(spl_lrnph, exist_ok=True) # 学習データ

    # 元データパス取得
    dir_lists = os.listdir(dir_ph) # カテゴリー毎のディレクトリパス
    # 隠しファイルを除外
    dir_lists = [f for f in dir_lists if os.path.isdir(os.path.join(dir_ph, f))]

    # コピー先のディレクトリ作成
    ph_lst = []  # 元データのカテゴリー毎のディレクトリパスを格納
    t_phlst = [] # カテゴリー毎のディレクトリパスを格納(テストデータ)
    l_phlst = [] # カテゴリー毎のディレクトリパスを格納(学習データ)

    # ディレクトリパス格納処理
    for d_name in dir_lists: # d_name:カテゴリー名のパス
        t_ph = os.path.join(spl_tstph, d_name)
        l_ph = os.path.join(spl_lrnph, d_name)
        os.makedirs(t_ph, exist_ok=True)
        os.makedirs(l_ph, exist_ok=True)
        t_phlst.append(t_ph)
        l_phlst.append(l_ph)
        ph_lst.append(os.path.join(dir_ph, d_name))

    # 分割処理
    for i,path in enumerate(ph_lst): # i: index番号, path:カテゴリー毎のディレクトリパス
        f_name = os.listdir(path)   # カテゴリー内の画像ファイルパス取得
        random.shuffle(f_name)      # ランダムに並び替え
        rate_ind = int(len(f_name)*rate) # 分割の割合分のindex数

        # 元データから学習データとして別ディレクトリに複製
        for l_fn in f_name[rate_ind:]:
            ori = os.path.join(path, l_fn)
            cpy = os.path.join(l_phlst[i], l_fn)
            shutil.copyfile(ori, cpy)

        # 元データからテストデータとして別ディレクトリに複製
        for t_fn in f_name[:rate_ind]:
            ori = os.path.join(path, t_fn)
            cpy = os.path.join(t_phlst[i], t_fn)
            shutil.copyfile(ori, cpy)
    print("データ分割完了")
    # カテゴリー毎に複製したテストデータ(t_phlst)と学習データ(l_phlst)のディレクトリパス
    return t_phlst, l_phlst

In [4]:
# 学習データ処理
def Load_LrnFile(path, ct_n): # path:学習データのディレクトリパス, ct_n:カテゴリー数
    img_data = [] # 画像の数値データ格納
    cate = []     # カテゴリーのindex格納
    cate_dct = {} # カテゴリーの並び順を格納

    for typ_nm, t_dr in enumerate(path): # t_dr:カテゴリー毎のディレクトリパス
        bs_nm = os.path.basename(t_dr) # カテゴリー名
        cate_dct[bs_nm] = typ_nm       # index格納
        f_lst = glob.glob(t_dr + "/*.png") # ディレクトリ内の全ファイルパス取得
        for fn in f_lst:
            img = load_img(fn) # 画像読み込み
            img_arr = img_to_array(img) # 画像データを行列の数値データに変換
            img_data.append(img_arr) # 格納
            cate.append(typ_nm)      # カテゴリーのindex格納
    img_data = np.array(img_data)    # ndarrayに変換
    cate = np.array(cate)
    img_data = img_data / 255.0      # 0.0〜1.0の範囲に正規化
    cate = to_categorical(cate, ct_n) # 識別用のarrayに変換
    IP_shape = img_data[0].shape  # 画像データのサイズ
    return img_data, cate, cate_dct, IP_shape

In [5]:
# テストデータ処理
def Load_TestFile(path): # path:テストデータのディレクトリパス
    img_data = [] # 画像の数値データ格納
    file_nm = []  # ファイル名格納
    for t_dr in path:
        f_lst = glob.glob(t_dr + "/*.png") # ディレクトリ内の全ファイルパス取得
        for fn in f_lst:
            img = load_img(fn) # 画像読み込み
            img_data.append(img_to_array(img)) # 画像データをndarrayの数値データに変換
            file_nm.append(fn)
    img_data = np.array(img_data)
    file_nm = np.array(file_nm)
    img_data = img_data / 255.0 # 正規化
    return img_data, file_nm

In [14]:
# スペクトログラム_CNN学習
# cate_n:カテゴリー数, lrn_rate:学習率, epo:エポック数, lrn_rate:学習データのディレクトリパス
# mdl_ph:学習したmodelを保存するファイルパス
def cnn_lrn(cate_n, lrn_rate, epo, lrn_path, mdl_ph):
    img_data, cate, c_dct, IP_shape = Load_LrnFile(lrn_path, 4) #学習データ処理
    model = keras.Sequential() # model宣言

    # 畳み込みでカーネルを用いてテンソルを生成する層
    conv = keras.layers.Conv2D(filters=16, kernel_size=(5,5), padding='same', activation='relu', input_shape=IP_shape)
    # プーリング層を追加
    pool = keras.layers.MaxPooling2D(pool_size=(2, 2))
    # データを平滑化(ノイズを目立たなくする)
    flttn = keras.layers.Flatten()
    # 全結合層
    ct_Lyr = keras.layers.Dense(units = cate_n, activation='softmax') # units: 出力数=カテゴリー数, 'softmax'を使用

    # 層追加
    model.add(conv)
    model.add(pool)
    model.add(flttn)
    model.add(ct_Lyr)

    # 確率的勾配降下法
    sgd = keras.optimizers.SGD(learning_rate=lrn_rate)
    # model作成
    model.compile(optimizer=sgd, loss="categorical_crossentropy")
    # modelの要約出力
    model.summary()

    #学習開始
    model.fit(img_data, cate, epochs=epo)
    # model保存(.h5)
    model.save(mdl_ph)
    print("学習完了")
    return c_dct

In [7]:
# 識別率を求める
# n_l:カテゴリー名, alsn:種類ごとのデータ総数を格納した行列, altn:種類ごとの識別数を格納した行列, csv_ph:csvファイルパス
def dsc_em(n_l, alsn, altn, csv_ph):
    Rslt_all = 0 # 全体の識別率

    Rslt = altn/alsn # 種類毎の識別率

    # 全体の識別率を求める
    for r in range(len(Rslt)):
        Rslt_all = Rslt_all + Rslt[r][r]
    Rslt_all = Rslt_all/len(Rslt)

    # 種類毎の識別率を.csvに保存 (index:実際の値, column:識別された値)
    R = pd.DataFrame(Rslt, index=n_l, columns=n_l)
    R.to_csv(csv_ph)

    # 結果表示
    print("識別率\n" + str(Rslt_all*100) + "%")
    print(R)
    print()

In [8]:
# 識別データ出力
# csv_ph:識別結果を保存したcsvファイルパス, arg_nl:カテゴリー名を格納したlist
def rslt_out(csv_ph, arg_nl):
    arg_len = len(arg_nl) # カテゴリー数取得
    jdg = pd.read_csv(csv_ph, index_col=[0]) # csv読み込み
    rslt_altn = np.zeros((arg_len, arg_len)) # 種類毎の識別数の行列
    rslt_alsn = np.zeros((arg_len, arg_len)) # 種類毎の全データ数の行列
    for i_1 in range(len(arg_nl)):
        rslt_tn = []
        rslt_sn = []
        for i_2 in arg_nl:
            sn = (jdg["actual"]==arg_nl[i_1]).sum()
            tn = ((jdg["actual"]==arg_nl[i_1])&(jdg["discrimination"]==i_2)).sum()
            rslt_tn.append(tn)
            rslt_sn.append(sn)
        rslt_altn[i_1] = rslt_altn[i_1] + rslt_tn
        rslt_alsn[i_1] = rslt_alsn[i_1] + rslt_sn
    return rslt_altn, rslt_alsn

In [9]:
# np.zeros配列set
def arr_zero(ln_0, ln_1):
    allt=np.zeros((ln_0,ln_1))
    alls=np.zeros((ln_0,ln_1))
    return allt, alls

In [10]:
# 識別結果保存(.csv)
# model_ph: modelを保存したファイルパス, tst_path:テストデータのディレクトリパス, cate_dct:カテゴリーの並び順を格納
def cnn_dsc(model_ph, tst_path, cate_dct):
    csv = "CNN_result.csv"  # 結果保存するファイルパス
    csv_ph = D_path + "/" + csv
    model = keras.models.load_model(model_ph) # model読み込み

    tst_dlst, fn_lst = Load_TestFile(tst_path) # テストデータ処理
    rslt = model.predict(tst_dlst) # 識別
    rslt_df = pd.DataFrame(columns=["discrimination", "actual", "TorF"]) # 識別結果格納DataFrame
    # 格納
    for r_i in range(len(rslt)):
        r_ind = np.argmax(rslt[r_i]) # 識別結果の値のindex取得
        dsc = [k for k, v in cate_dct.items() if v == r_ind][0] # indexを用いて識別結果取得
        typ_nm = os.path.basename(os.path.dirname(fn_lst[r_i])) # ファイル名取得
        # 格納
        rslt_df.loc[os.path.basename(fn_lst[r_i])] = [dsc, typ_nm, bool(dsc == typ_nm)]
    # 識別結果保存
    rslt_df.to_csv(csv_ph)
    return csv_ph

In [11]:
mdl = "lrn_model.h5"
cnn_Lrnrate = 0.01        # 学習率
cnn_epo = 12             # epoch数
Categories_name = ['ebony', 'overseas', 'maple_tree', 'ebony+leather']

tst_ph, lrn_ph = Test_Set_CNN(0.1)   # 学習データとテストデータ分割

データ分割完了


In [15]:
cnn_epo = 12
cate_dct = cnn_lrn(4, cnn_Lrnrate, cnn_epo, lrn_ph, mdl) # 学習

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 240, 320, 16)      1216      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 120, 160, 16)     0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 307200)            0         
                                                                 
 dense_1 (Dense)             (None, 4)                 1228804   
                                                                 
Total params: 1,230,020
Trainable params: 1,230,020
Non-trainable params: 0
_________________________________________________________________
Epoch 1/12
57/57 [==============================] - 119s 2s/step - loss: 2.4018
Epoch 2/12
57/57 [============

In [16]:
csv_cnn = cnn_dsc(mdl, tst_ph, cate_dct)   # 識別
altn_cnn, alsn_cnn = rslt_out(csv_cnn, Categories_name)  # 識別結果保存

# 識別結果表示
print()
print("CNN")
dsc_em(Categories_name, alsn_cnn, altn_cnn, "/content/drive/MyDrive/c_result.csv")

7/7 [==============================] - 6s 838ms/step

CNN
識別率
79.08695652173911%
                  ebony  overseas  maple_tree  ebony+leather
ebony          1.000000  0.000000    0.000000       0.000000
overseas       0.120000  0.860000    0.000000       0.020000
maple_tree     0.391304  0.043478    0.543478       0.021739
ebony+leather  0.240000  0.000000    0.000000       0.760000



In [ ]:
#ファイル数確認のためのコード
s_path = os.path.join(D_path,'CNN_sp_data/CNN_train_data/ebony')

ss = glob.glob(s_path + '/**')

print(s_path)
print(len(ss))

NameError: name 'os' is not defined